In [591]:
#install nfl_data_py package
!pip install nfl_data_py


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [592]:
import nfl_data_py as nfl


In [593]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tqdm import tqdm

import nfl_data_py as nfl

%matplotlib inline

In [594]:
from sklearn.metrics import mean_absolute_error

In [595]:
#Calculate the market-implied team total, derived the betting lines

def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [596]:
def last_name(full):
  if full == 'DanielThomas':
    return 'Thomas'
  elif full == 'JulioJones':
    return 'Jones'
  elif '.' not in full:
    print(full)
    return full
  else:
    return full.split(".",1)[1]

In [597]:
def team_fixer(team):
    if team == 'SD':
        return 'LAC'
    elif team == 'OAK':
        return 'LV'
    elif team == 'STL':
        return 'LA'
    else:
        return team

In [598]:
def snap_name_changer(player):
    if 'Jr.' in player:
        return player.split(' Jr.')[0]
        #return 'D.J. Davis'
    elif 'II' in player:
        return player.split(' II')[0]
    else:
        return player

In [599]:
def position_handler(pos):
  if pos in ['WR','RB','TE','FB']:
    return pos
  elif pos == 'RB/W':
    return 'RB'
  elif pos == 'WR/R':
    return 'WR'
  elif pos == 'HB':
    return 'RB'
  else:
    return 'other'

In [600]:
import joblib
import pickle


play_by_play_model = pickle.load(open('play_by_play.pkl', 'rb'))


In [601]:
in_season_model = pickle.load(open('model.pkl', 'rb'))

In [602]:
linear_model = pickle.load(open('linear_model.pkl', 'rb'))

# Apply Model

In [603]:
#load in data for NFL pass attempts

szn_receivers=[]
target_list=[]
  
df_list = []
weekly_list = []
target_week = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
week_list1= []


all_throws = []
n_szn = []
plus_1_szn = []

for YEAR in tqdm(range(2022,2023)):
    print(YEAR)
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','week','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch']]

    current_szn_df = df.dropna(subset=['receiver_player_name'])

  #Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 

    current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go']].set_index(['receiver_player_name','posteam'])
    current_szn_df['year'] = YEAR
  #apply model to the season's stats
    current_x = current_szn_df.drop(['yards_gained','year','week','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()
  
    
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    weekly_yac = yac_df.groupby(['receiver_player_name','posteam','week']).agg({'complete_pass':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})
    
    
    
    weekly_pass_offense=current_szn_df.groupby(['posteam','week']).agg({'week':'count','air_yards':'sum'})
    weekly_pass_offense.rename(columns={'week':'team_attempts','air_yards':'team_air_yards'},inplace=True)
    team_attempts=weekly_pass_offense.reset_index()[['posteam','week','team_attempts','team_air_yards']]
    team_attempts.set_index(['posteam','week'],inplace=True)
    weekly_rec_group = current_szn_df.groupby(['receiver_player_name','posteam','week']).agg({'week':'max','down':'count','complete_pass':'sum','yards_gained':'sum','air_yards':'sum','xYards':'sum','cp':'sum','implied_posteam_total':'max','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum'})
    weekly_rec_group.rename(columns={'week':'week#','down':'targets'},inplace=True)
    weekly_rec_group = weekly_rec_group.merge(weekly_yac[['yards_after_catch','xyac_mean_yardage']],right_index=True,left_index=True)
    
    
    feature_receivers = weekly_rec_group.reset_index().set_index(['posteam','week']).merge(team_attempts,right_index=True,left_index=True)
    feature_receivers = feature_receivers.reset_index().set_index(['receiver_player_name','posteam','week'])
    feature_receivers['szn'] = YEAR
    
    week_df1 = feature_receivers.copy()
    week_df = week_df1.copy()
    week_df = week_df.reset_index().set_index(['receiver_player_name','posteam'])
    feature_df = feature_receivers.copy()
    
    for target in tqdm(target_week):
        feature_receivers = feature_df[feature_df['week#']<target]
    
        player_list = []

        for x in feature_receivers.reset_index().set_index(['receiver_player_name','posteam']).index:
            player_list.append(x)
    
    
        players = list(set(player_list))

        print(target)
        for player in players:
            feature_receivers['xYards_median'] = feature_receivers['xYards']
            weekly_player = pd.DataFrame(feature_receivers.loc[player].sort_values('week#',ascending=False).head(6).agg({'yards_gained':'sum','targets':'sum','complete_pass':'sum','week#':'count','szn':'max','air_yards':'sum','xYards':'sum','cp':'sum','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum','team_attempts':'sum','team_air_yards':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum','xYards_median':'median'})).transpose()
            weekly_player['player'] = player[0]
            weekly_player['team'] = player[1]
            weekly_player['target_week'] = target
            weekly_player.set_index(['player','team','target_week','szn'],inplace=True)
            
          
            df_list.append(weekly_player)
        
        week_df = week_df1.copy()
        
        week_df = week_df[week_df['week#']==target]
        week_df = week_df.reset_index().rename(columns={'week':'target_week','receiver_player_name':'player','posteam':'team'})
        week_df = week_df.set_index(['player','team','target_week','szn'])

        week_list1.append(week_df)
        
        
        
        
        
weekly_receivers = pd.concat(df_list)
weekly_targets = pd.concat(week_list1)
        
        #weekly_receivers['aDOT']=weekly_receivers['air_yards']/weekly_receivers['targets']
        #weekly_receivers['AY_share']=round(weekly_receivers['air_yards']/weekly_receivers['team_air_yards'],3)
        #weekly_receivers['target_share']=round(weekly_receivers['targets']/weekly_receivers['team_attempts'],3)
        #weekly_receivers['yac/rec']=round(weekly_receivers['yards_after_catch']/weekly_receivers['complete_pass'],3)
        #weekly_receivers['xyac/rec']=round(weekly_receivers['xyac_mean_yardage']/weekly_receivers['complete_pass'],3)


        
        

  0%|                                                     | 0/1 [00:00<?, ?it/s]

2022



  0%|                                                    | 0/18 [00:00<?, ?it/s]

1
2



 11%|████▉                                       | 2/18 [00:16<02:08,  8.02s/it]

...


 89%|██████████████████████████████████████▏    | 16/18 [04:56<00:33, 16.99s/it]

17



 94%|████████████████████████████████████████▌  | 17/18 [05:03<00:13, 13.98s/it]

18



100%|████████████████████████████████████████████| 1/1 [06:13<00:00, 373.80s/it]


In [604]:
feature_df['target_week'] = feature_df['week#']+1

feature_df.drop(('A.St. Brown','DET',7),inplace=True)

In [605]:
feature_df.loc['T.Lockett']

week#  targets  complete_pass  yards_gained  air_yards  \
posteam week                                                           
SEA     1         1        4            3.0          28.0       29.0   
        2         2       11            9.0         107.0      118.0   
        3         3       11            9.0          76.0      123.0   
        4         4        8            6.0          91.0      122.0   
        5         5        6            5.0         104.0      135.0   
        6         6        5            2.0          17.0       62.0   
        7         7        8            7.0          45.0       43.0   
        8         8        8            5.0          63.0       84.0   
        9         9        5            5.0          67.0       24.0   
        10       10        5            3.0          42.0       53.0   
        12       12        7            3.0          68.0       90.0   
        13       13       12            9.0         128.0       89.0   

                 xYards        cp  implied_posteam_total  goal_to_go  qb_hit  \
posteam week                                                                   
SEA     1     29.970425  2.799239                  19.00           0     0.0   
        2     80.492153  6.903809                  15.75           0     2.0   
        3     84.512757  6.664902                  22.00           0     1.0   
        4     65.110391  4.626942                  22.75           1     2.0   
        5     51.207771  3.103960                  20.00           0     2.0   
        6     39.778139  3.077915                  23.75           0     0.0   
        7     52.408489  5.625344                  23.00           0     1.0   
        8     51.552411  4.539161                  23.75           0     2.0   
        9     31.774134  3.406899                  24.00           1     1.0   
        10    37.522324  3.040777                  21.25           0     0.0   
        12    64.875953  4.536230                  25.75           0     1.0   
        13    89.451189  8.285791                  24.00           1     1.0   

              middle  yards_after_catch  xyac_mean_yardage  team_attempts  \
posteam week                                                                
SEA     1          1                5.0          15.557277             27   
        2          3               22.0          36.641333             30   
        3          0                3.0          34.271528             44   
        4          2               11.0          22.844684             29   
        5          2                5.0          10.438336             24   
        6          2                8.0           7.939587             30   
        7          1                9.0          34.396273             26   
        8          0               28.0          27.092477             33   
        9          1               43.0          27.223590             32   
        10         1                9.0           8.453819             33   
        12         5               24.0           9.173747             37   
        13         2               68.0          43.275493             37   

              team_air_yards   szn  target_week  
posteam week                                     
SEA     1              135.0  2022            2  
        2              170.0  2022            3  
        3              412.0  2022            4  
        4              319.0  2022            5  
        5              300.0  2022            6  
        6              204.0  2022            7  
        7              209.0  2022            8  
        8              261.0  2022            9  
        9               75.0  2022           10  
        10             248.0  2022           11  
        12             294.0  2022           13  
        13             270.0  2022           14

In [606]:
    YEAR=2022
    
    
    snap_df = nfl.import_snap_counts([YEAR])
    snap_df['team'] = snap_df['team'].apply(lambda x: team_fixer(x))
    snap_df = snap_df[snap_df['game_type']=='REG']
    snap_df = snap_df[['pfr_player_id','player','offense_snaps','offense_pct','week','team','game_id','position']]
    snap_df['player_copy'] = snap_df['player']
    snap_df.rename(columns={'player_copy':'full_name'},inplace=True)
    snap_df['full_name'] = snap_df['full_name'].apply(lambda x: snap_name_changer(x))
    
    
    snap_df['week#'] = snap_df['week']
    #snap_df = snap_df.set_index(['receiver_player_id','team','week'])
    snap_df = snap_df[snap_df['offense_snaps']>0]

    snap_df['last_name'] = snap_df['full_name'].apply(lambda x: x.split(" ",1)[1])
    snap_df['identifier'] = (snap_df['full_name']).str[0]+"."+snap_df['last_name']+"_"+snap_df['team']+"_"+(snap_df['week'].astype(str))
    snap_df = snap_df[['full_name','identifier','offense_snaps','position','team']]




In [607]:
snap_df[snap_df['team']=='IND']['full_name'].value_counts()

Ryan Kelly          13
Parris Campbell     13
Mo Alie-Cox         13
Quenton Nelson      13
Braden Smith        12
Michael Pittman     12
Kylen Granson       12
Alec Pierce         12
Danny Pinter        11
Matt Ryan           11
Matt Pryor          11
Jelani Woods        11
Mike Strachan       10
Jonathan Taylor     10
Deon Jackson         9
Bernhard Raimann     9
Ashton Dulin         9
Nyheim Hines         7
Will Fries           7
Dennis Kelly         4
Zack Moss            4
Phillip Lindsay      3
Nikola Kalinic       3
Sam Ehlinger         2
Jordan Wilkins       2
Keke Coutee          2
Dezmon Patmon        1
Nick Foles           1
Ethan Fernea         1
Name: full_name, dtype: int64

In [608]:
snap_df = snap_df[snap_df['identifier']!='A.St. Brown_DET_7']

snap_df = snap_df[snap_df['identifier']!='D.Metcalf_SEA_7']

In [609]:
snap_df = snap_df[snap_df['identifier']!='M.Andrews_BAL_8']


In [610]:
from tqdm import tqdm
import tqdm.notebook as tq
import warnings
warnings.filterwarnings('ignore')

In [611]:
player_df_list = []



for player_id in tq.tqdm(feature_df.index):

  player = player_id[0:2]
  player_df = feature_df.loc[player]
  week_df=pd.DataFrame()
  week_df['week#']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
  player_df = player_df.merge(week_df,how='right')
  player_df['receiver_player_name'] = player[0]
  
  player_df['posteam'] = player[1]
  player_df['last'] = player_df['receiver_player_name'].apply(lambda x: last_name(x))
  player_df['identifier'] = (player_df['receiver_player_name']).str[0]+"."+player_df['last']+"_"+player_df['posteam']+"_"+player_df['week#'].astype(str)
  player_df = player_df.merge(snap_df,on='identifier')

  if player == 'A.St. Brown':
    player_df = player_df


  player_df.fillna(0,inplace=True)

  player_df['xYards_median'] = player_df['xYards'].rolling(6,min_periods=6).median()
  player_df['xYards/game'] = player_df['xYards'].rolling(6,min_periods=6).mean()

  player_df['xYards_sum'] = player_df['xYards'].rolling(6,min_periods=6).sum()
  player_df['yards_sum'] = player_df['yards_gained'].rolling(6,min_periods=6).sum()
  player_df['yards/game'] = player_df['yards_gained'].rolling(6,min_periods=6).mean()
  player_df['games'] = player_df['week#'].rolling(6,min_periods=1).count()
  player_df.rename(columns={'week#':'week'},inplace=True)
  target_df1 = pd.DataFrame()


  player_df['target_week'] = player_df['week']+1

  target_df1['target_week']=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

  player_df = player_df.merge(target_df1,how='right').sort_values('target_week')




  player_df = player_df.fillna(method='ffill')

  player_df_list.append(player_df)




  0%|          | 0/2774 [00:00<?, ?it/s]

In [612]:
trailing_weeks = pd.concat(player_df_list)


In [613]:
player_analyzer = trailing_weeks.drop_duplicates()

player_analyzer['aDOT'] = player_analyzer['air_yards']/player_analyzer['targets']
player_analyzer['target_share'] = player_analyzer['targets']/player_analyzer['team_attempts']




In [614]:
trailing_weeks = trailing_weeks.dropna()

trailing_weeks['games'].value_counts()

6.0    28733
Name: games, dtype: int64

In [615]:
trailing_weeks = trailing_weeks.drop_duplicates()

In [616]:
trailing_weeks['position'] = trailing_weeks['position'].apply(lambda x: position_handler(x))

In [617]:
one_hot = pd.get_dummies(trailing_weeks['position'])
one_hot.reset_index(inplace=True)
trailing_weeks = trailing_weeks.drop('position',axis = 1).reset_index()

trailing_weeks = trailing_weeks.join(one_hot,how='left', lsuffix='_left', rsuffix='_right')

#drop non WR/TE/RB/FB
trailing_weeks = trailing_weeks[trailing_weeks['other']==0]

In [618]:
trailing_weeks_all = trailing_weeks.copy()

In [619]:
np.mean([76.6,76.9,62.4,32,75.1,79.4])

67.06666666666666

## 5. Actual Results

### Week 8

In [620]:
trailing_weeks_all.rename(columns={'yards/game':'Yards/game'},inplace=True)

trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==8]

In [621]:
#trailing_weeks = weekly_receivers.reset_index()
#trailing_weeks = trailing_weeks[trailing_weeks['target_week']==8]



#trailing_weeks['xYards/game'] = trailing_weeks['xYards']/trailing_weeks['games']
#trailing_weeks['Yards/game'] = trailing_weeks['yards_gained']/trailing_weeks['games']
#trailing_weeks['target_share'] = trailing_weeks['targets']/trailing_weeks['team_attempts']
#trailing_weeks['AY_share'] = trailing_weeks['air_yards']/trailing_weeks['team_air_yards']
#trailing_weeks['yac/rec'] = trailing_weeks['yards_after_catch']/trailing_weeks['complete_pass']
#trailing_weeks['xyac/rec'] = trailing_weeks['xyac_mean_yardage']/trailing_weeks['complete_pass']


#trailing_weeks['aDOT']=trailing_weeks['air_yards']/trailing_weeks['targets']
#trailing_weeks['Yards/target'] = trailing_weeks['yards_gained']/trailing_weeks['targets']
#trailing_weeks['xYards/target'] = trailing_weeks['xYards']/trailing_weeks['targets']

#trailing_weeks.rename(columns={'AY_share':'AY_share_x','target_share':'target_share_x','yards_gained':'yards_gained_x','xYards':'xYards_x','Yards/target':'Yards/target_x','xYards/target':'xYards/target_x','team_attempts':'team_attempts_x','catch_rate':'catch_rate_x'},inplace=True)


#trailing_weeks['team_attempts/game_x'] = trailing_weeks['team_attempts_x']/trailing_weeks['games']


#trailing_weeks['targets/game_x'] = trailing_weeks['targets']/trailing_weeks['games']

#trailing_weeks['catch_rate_x'] = trailing_weeks['complete_pass']/trailing_weeks['targets']
#trailing_weeks['yac/rec_x'] = trailing_weeks['yards_after_catch']/trailing_weeks['complete_pass']


#trailing_weeks['cpoe_x'] = trailing_weeks['complete_pass']-trailing_weeks['cp']
#trailing_weeks['yacoe_x'] = trailing_weeks['yac/rec'] - trailing_weeks['xyac/rec']
#trailing_weeks['complete_pass_x'] = trailing_weeks['complete_pass']
#trailing_weeks['complete_pass_x'] = trailing_weeks['complete_pass']
#trailing_weeks['aDOT_x'] = trailing_weeks['aDOT']
#trailing_weeks['catches_per_game_x'] = trailing_weeks['complete_pass']/trailing_weeks['games']

In [622]:
trailing_weeks.dropna(inplace=True)

In [623]:
trailing_weeks

,index_left,week,targets,complete_pass,yards_gained,air_yards,xYards,cp,implied_posteam_total,goal_to_go,...,xYards_sum,yards_sum,Yards/game,games,index_right,FB,RB,TE,WR,other
11,6,7.0,5.0,4.0,21.0,-11.0,23.951324,4.043840,23.25,1.0,...,104.496487,113.0,18.833333,6.0,6,0,1,0,0,0
23,6,7.0,1.0,1.0,5.0,4.0,2.921464,0.659675,23.25,1.0,...,98.328598,146.0,24.333333,6.0,6,0,0,0,1,0
44,6,6.0,9.0,5.0,68.0,151.0,102.644805,5.008422,26.25,0.0,...,539.368438,485.0,80.833333,6.0,6,0,0,0,1,0
56,6,7.0,4.0,2.0,21.0,28.0,31.020628,2.912860,23.25,1.0,...,369.269097,306.0,51.000000,6.0,6,0,0,1,0,0
73,6,7.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,5.805800,2.0,0.333333,6.0,6,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,6,7.0,8.0,5.0,73.0,96.0,57.285837,4.606122,18.75,0.0,...,335.313037,382.0,63.666667,6.0,6,0,0,0,1,0
3876,6,7.0,1.0,1.0,4.0,1.0,3.224259,0.597253,18.75,0.0,...,22.639310,27.0,4.500000,6.0,6,0,0,0,1,0
3887,6,7.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,73.811671,73.0,12.166667,6.0,6,0,0,1,0,0
3899,6,6.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,...,82.495672,111.0,18.500000,6.0,6,0,0,0,1,0


In [624]:
features = ['xYards/game','Yards/game','RB','WR','FB']

linear_features = ['xYards/game','Yards/game']

In [625]:
prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['model_average']=(trailing_weeks['linear_projection']+trailing_weeks['gb_projection'])/2
trailing_weeks['pYards']=trailing_weeks['gb_projection']


In [626]:
projections = trailing_weeks.sort_values('pYards',ascending=False)

#[['player','team','pYards','Yards/game','games','target_share_x','aDOT','xYards/game','team_attempts_x']]

In [627]:
projections[projections['receiver_player_name']=='M.Andrews']

,index_left,week,targets,complete_pass,yards_gained,air_yards,xYards,cp,implied_posteam_total,goal_to_go,...,index_right,FB,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards
318,6,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6,0,0,1,0,0,67.571468,63.480619,65.526044,67.571468


In [628]:
df=pd.read_csv('betting_lines/nfl-best-bets.csv')
df = df[df['team']!='CLV']

mnf=pd.read_csv('betting_lines/nfl-best-bets (1).csv')

In [629]:
df = df.append(mnf)

In [630]:
projections.reset_index(inplace=True)

projections.rename(columns={'receiver_player_name':'player'},inplace=True)

In [631]:
names_to_change = ['M.Jones Jr','E.St','I.Smith Jr','A.St','T.Marshall Jr','A.Robinson II','M.Pittman Jr','K.Walker III','M.Gordon III']

def name_changer(name):
    if name in names_to_change:
        if name == 'M.Jones Jr':
            return 'M.Jones'
        if name == 'E.St':
            return 'E.St. Brown'
        if name == 'I.Smith Jr':
            return 'I.Smith'
        if name == 'A.St':
            return 'A.St. Brown'
        if name == 'T.Marshall Jr':
            return 'T.Marshall'
        if name == 'A.Robinson II':
            return 'Al.Robinson'
        if name == 'M.Pittman Jr':
            return 'M.Pittman'
        if name == 'K.Walker III':
            return 'K.Walker'
        if name == 'M.Gordon III':
            return 'M.Gordon'
    if name not in names_to_change:
        return name

In [632]:
odds = df[['player','team','line']]

def team_changer(name):
    if name == 'HST':
        return 'HOU'
    elif name == 'CLV':
        return 'CLE'
    elif name == 'ARZ':
        return 'ARI'
    elif name == 'BLT':
        return 'BAL'
    else:
        return name

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))



table = odds.merge(projections,how='left')

table = table.round(2)

table['diff'] = table['pYards'] - table['line']




table['abs_diff'] = abs(table['diff'])



In [633]:
table.sort_values('abs_diff',ascending=False).head(50)

,player,team,line,index,index_left,week,targets,complete_pass,yards_gained,air_yards,...,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards,diff,abs_diff
8,M.Jones,JAX,28.5,1806.0,6.0,7.0,8.0,4.0,57.0,122.0,...,0.0,0.0,1.0,0.0,48.84,47.62,48.23,48.84,20.34,20.34
80,C.Kupp,LA,92.5,2078.0,6.0,6.0,8.0,7.0,80.0,37.0,...,0.0,0.0,1.0,0.0,73.47,83.51,78.49,73.47,-19.03,19.03
89,M.Pittman,IND,57.5,1680.0,6.0,7.0,9.0,6.0,58.0,68.0,...,0.0,0.0,1.0,0.0,71.29,69.06,70.17,71.29,13.79,13.79
58,M.Callaway,NO,39.5,2798.0,6.0,7.0,6.0,2.0,20.0,85.0,...,0.0,0.0,1.0,0.0,26.81,25.27,26.04,26.81,-12.69,12.69
24,J.Jefferson,MIN,89.5,2539.0,6.0,6.0,8.0,6.0,107.0,41.0,...,0.0,0.0,1.0,0.0,76.87,80.55,78.71,76.87,-12.63,12.63
52,T.Conklin,NYJ,19.5,3040.0,6.0,7.0,6.0,4.0,22.0,45.0,...,0.0,1.0,0.0,0.0,31.23,32.23,31.73,31.23,11.73,11.73
73,B.Cooks,HOU,60.5,1531.0,6.0,7.0,5.0,4.0,46.0,44.0,...,0.0,0.0,1.0,0.0,48.84,47.32,48.08,48.84,-11.66,11.66
119,T.Higgins,CIN,76.5,843.0,6.0,7.0,7.0,5.0,93.0,76.0,...,0.0,0.0,1.0,0.0,65.36,59.21,62.28,65.36,-11.14,11.14
28,J.Reynolds,DET,39.5,1267.0,6.0,7.0,2.0,1.0,8.0,41.0,...,0.0,0.0,1.0,0.0,50.35,49.60,49.97,50.35,10.85,10.85
40,D.Moore,CAR,60.5,527.0,6.0,7.0,10.0,7.0,69.0,88.0,...,0.0,0.0,1.0,0.0,49.77,47.66,48.72,49.77,-10.73,10.73


In [634]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

  

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.32s/it]


In [635]:
#table.shape[0]


table = table.drop_duplicates()

In [636]:
table['pYards'].corr(table['line'])


0.9532328928893034

In [637]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==8]



100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.61s/it]


In [638]:
weekly_Rec = results.groupby('receiver_player_name').agg({'yards_gained':'sum','pass':'count','complete_pass':'sum'}).reset_index().rename(columns={'receiver_player_name':'player'})

In [639]:
table.drop(columns={'yards_gained'},inplace=True)

In [640]:
smash_table = table[table['abs_diff']>=10]

results = table.merge(weekly_Rec,on='player')


In [641]:
results=results.dropna()



In [642]:
results['Over_Picked'] =results['pYards']>results['line']

In [643]:
results['Over_Result'] = results['yards_gained'] > results['line']

results['Correct'] = results['Over_Picked'] == results['Over_Result']

In [644]:
#results = results[results['games']>1]



In [645]:
results['xYards/game'] = results['xYards_sum']/results['games']

In [646]:
w8_results = results[['player','team','games','line','pYards','xYards_median','xYards/game','yards_gained','Over_Picked','Over_Result','Correct','gb_projection','linear_projection']]

w8_results['week'] = 8

w8_results = w8_results[['player','team','games','line','pYards','xYards_median','xYards/game','yards_gained','Over_Picked','Over_Result','Correct','gb_projection','linear_projection']]


In [647]:
results['yards_gained'].corr(results['pYards'])

0.6274678666725377

In [648]:
#0.5860590594191387

In [649]:
import math
MSE = np.square(np.subtract(results['yards_gained'],results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (players w/ listed odds):\n")
print(RMSE)

print('-------------')

weekly_all  = weekly_Rec.merge(projections)[['player','pYards','yards_gained']]

MSE = np.square(np.subtract(weekly_all['yards_gained'],weekly_all['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (all players):\n")
print(RMSE)

Root Mean Square Error (players w/ listed odds):

30.665086336092386
-------------
Root Mean Square Error (all players):

21.851090528835137


In [650]:
#30.880608939504032
#-------------

#30.24493362852292

In [651]:
#overall
results['Correct'].value_counts()

True     52
False    48
Name: Correct, dtype: int64

In [652]:
smash_results = results[results['abs_diff']>=10]

smash_results['Correct'].value_counts()

True     8
False    7
Name: Correct, dtype: int64

In [653]:
smash_results[['player','line','xYards/game','pYards','abs_diff','yards_gained','Over_Picked','Over_Result','Correct']].sort_values('abs_diff',ascending=False)

,player,line,xYards/game,pYards,abs_diff,yards_gained,Over_Picked,Over_Result,Correct
8,M.Jones,28.5,55.080000,48.84,20.34,0.0,True,False,False
78,C.Kupp,92.5,86.716667,73.47,19.03,79.0,False,False,True
87,M.Pittman,57.5,73.588333,71.29,13.79,53.0,True,False,False
57,M.Callaway,39.5,28.781667,26.81,12.69,10.0,False,False,True
23,J.Jefferson,89.5,76.630000,76.87,12.63,98.0,False,True,False
51,T.Conklin,19.5,31.301667,31.23,11.73,79.0,True,True,True
71,B.Cooks,60.5,53.315000,48.84,11.66,73.0,False,True,False
116,T.Higgins,76.5,59.608333,65.36,11.14,49.0,False,False,True
27,J.Reynolds,39.5,50.543333,50.35,10.85,14.0,True,False,False
39,D.Moore,60.5,59.985000,49.77,10.73,152.0,False,True,False


In [654]:
picked_overs = results[results['Over_Picked']==True]

picked_overs['Correct'].value_counts()

True     34
False    30
Name: Correct, dtype: int64

In [655]:
picked_unders = results[results['Over_Picked']==False]

picked_unders['Correct'].value_counts()

True     18
False    18
Name: Correct, dtype: int64

In [656]:
results['Over_Result'].value_counts()

True     52
False    48
Name: Over_Result, dtype: int64

In [657]:
smash_overs = smash_results[smash_results['Over_Picked']==True]


smash_overs['Correct'].value_counts()

False    4
True     3
Name: Correct, dtype: int64

In [658]:
smash_unders = smash_results[smash_results['Over_Picked']==False]


smash_unders['Correct'].value_counts()

True     5
False    3
Name: Correct, dtype: int64

In [659]:
table[table['team']=='CLE']

,player,team,line,index,index_left,week,targets,complete_pass,air_yards,xYards,...,RB,TE,WR,other,gb_projection,linear_projection,model_average,pYards,diff,abs_diff
121,A.Cooper,CLE,53.5,887.0,6.0,7.0,4.0,3.0,84.0,35.33,...,0.0,0.0,1.0,0.0,63.78,62.37,63.08,63.78,10.28,10.28
122,D.Bell,CLE,15.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,D.Peoples-Jones,CLE,44.5,923.0,6.0,7.0,6.0,6.0,59.0,41.68,...,0.0,0.0,1.0,0.0,41.29,39.96,40.62,41.29,-3.21,3.21
124,K.Hunt,CLE,12.5,947.0,6.0,7.0,2.0,1.0,-6.0,11.01,...,1.0,0.0,0.0,0.0,13.19,13.88,13.53,13.19,0.69,0.69
125,N.Chubb,CLE,7.5,959.0,6.0,7.0,3.0,2.0,-8.0,15.41,...,1.0,0.0,0.0,0.0,11.35,12.32,11.83,11.35,3.85,3.85


### Week 9

In [660]:
#trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==9]


trailing_weeks = trailing_weeks.dropna()

prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']



In [661]:
trailing_weeks.drop_duplicates().shape[0]

283

In [662]:
w9_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards_median','gb_projection','linear_projection']]

In [663]:
w9_table.rename(columns={'receiver_player_name':'player'},inplace=True)

In [664]:
w9_table.set_index(['player','team'],inplace=True)

In [665]:
week_by_week = feature_receivers.reset_index()

week_by_week.set_index(['receiver_player_name','posteam']).loc['A.St. Brown'][['week','targets','yards_gained','xYards','complete_pass','cp',]]

,week,targets,yards_gained,xYards,complete_pass,cp
posteam,,,,,,
DET,1,12,64.0,74.672700,8.0,8.883887
DET,2,12,116.0,76.962425,9.0,7.904535
DET,3,9,73.0,62.434424,6.0,6.727337
DET,5,6,18.0,31.988359,4.0,4.080900
DET,7,1,4.0,5.330446,1.0,0.883786
DET,8,10,69.0,75.051747,7.0,7.245048
DET,9,9,55.0,79.371461,4.0,6.638146
DET,10,11,119.0,76.603081,10.0,8.014863
DET,11,8,76.0,51.823136,7.0,5.685872


In [666]:
week_by_week = feature_receivers.reset_index()

sun_g = week_by_week.set_index(['receiver_player_name','posteam']).loc['A.St. Brown'][['week','targets','yards_gained','xYards','complete_pass','cp',]]

sun_g = sun_g[sun_g['targets']>1]

sun_g['xYards_trail'] = sun_g['xYards'].rolling(6,min_periods=6).mean()

sun_g

,week,targets,yards_gained,xYards,complete_pass,cp,xYards_trail
posteam,,,,,,,
DET,1,12,64.0,74.672700,8.0,8.883887,NaN
DET,2,12,116.0,76.962425,9.0,7.904535,NaN
DET,3,9,73.0,62.434424,6.0,6.727337,NaN
DET,5,6,18.0,31.988359,4.0,4.080900,NaN
DET,8,10,69.0,75.051747,7.0,7.245048,NaN
DET,9,9,55.0,79.371461,4.0,6.638146,66.746852
DET,10,11,119.0,76.603081,10.0,8.014863,67.068583
DET,11,8,76.0,51.823136,7.0,5.685872,62.878701
DET,12,10,122.0,80.189604,9.0,7.363505,65.837898


In [667]:
w9_odds = pd.read_csv('betting_lines/nfl-best-bets (3).csv').round(1)[['player','team','line']]
w9_odds = w9_odds[w9_odds['team']!='BLT']
w9_odds = w9_odds[w9_odds['team']!='NO']


w9_odds = w9_odds.append(pd.read_csv('betting_lines/nfl-best-bets (4).csv').round(1)[['player','team','line']])



odds = w9_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


In [668]:
odds.set_index(['player','team'],inplace=True)

In [669]:

w9_table = w9_table.merge(odds,right_index=True,left_index=True)
w9_table = w9_table.reset_index()

w9_table['abs. diff'] = abs(w9_table['pYards'] - w9_table['line'])

w9_table = w9_table.round(1)

w9_table['Over_Picked'] =w9_table['pYards']>w9_table ['line']

#w9_table.sort_values('abs. diff',ascending=False).head(50)


In [670]:
w9_table[w9_table['team']=='BAL']

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked
4,D.Duvernay,BAL,6.0,33.4,26.2,33.4,32.1,42.5,9.1,False
5,K.Drake,BAL,6.0,9.2,5.9,9.2,9.0,12.5,3.3,False
6,I.Likely,BAL,6.0,19.6,10.0,19.6,21.2,41.5,21.9,False
7,J.Oliver,BAL,6.0,8.8,3.3,8.8,9.0,7.5,1.3,True


In [671]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==9]


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/it]


In [672]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w9_table = w9_table.merge(results)

w9_table['Over_Result'] = w9_table['yards_gained']>w9_table['line']

w9_table['Win'] = (w9_table['Over_Result'] == w9_table['Over_Picked'])*1



In [673]:
w9_table = w9_table[w9_table['games']>1]

In [674]:

import math
MSE = np.square(np.subtract(w9_table['pYards'],w9_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w9_table['yards_gained'],w9_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

27.447842716094133
-----
Root Mean Square Error - Betting Line:

24.991510186368266


In [675]:
w9_table['week'] = 9

In [676]:
w9_table

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
0,Z.Ertz,ARI,6.0,46.7,51.0,46.7,48.2,39.5,7.2,True,40.0,True,1,9
1,D.London,ATL,6.0,37.6,46.2,37.6,34.8,38.5,0.9,False,23.0,False,1,9
2,K.Pitts,ATL,6.0,38.6,36.1,38.6,38.3,40.5,1.9,False,27.0,False,1,9
3,O.Zaccheaus,ATL,6.0,30.6,25.4,30.6,30.4,29.5,1.1,True,19.0,False,0,9
4,D.Duvernay,BAL,6.0,33.4,26.2,33.4,32.1,42.5,9.1,False,5.0,False,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,N.Westbrook-Ikhine,TEN,6.0,19.7,13.7,19.7,18.5,24.5,4.8,False,0.0,False,1,9
82,R.Woods,TEN,6.0,37.7,34.0,37.7,36.1,38.5,0.8,False,0.0,False,1,9
83,A.Gibson,WAS,6.0,19.6,20.1,19.6,22.1,22.5,2.9,False,11.0,False,1,9
84,C.Samuel,WAS,6.0,47.1,50.5,47.1,43.1,48.5,1.4,False,65.0,True,0,9


In [677]:
w9_table = w9_table[['player','team','games','week','line','pYards','xYards_median','Over_Picked','yards_gained','Over_Result','Win','gb_projection','linear_projection']]

In [678]:
w8_results['Correct'] = w8_results['Correct']*1
w8_results.rename(columns={'Correct':'Win'},inplace=True)

In [679]:
w8_results[w8_results['team']=='DET']

,player,team,games,line,pYards,xYards_median,xYards/game,yards_gained,Over_Picked,Over_Result,Win,gb_projection,linear_projection
27,J.Reynolds,DET,6.0,39.5,50.35,41.79,50.543333,14.0,True,False,0,50.35,49.60
28,K.Raymond,DET,6.0,29.5,29.40,25.69,27.521667,76.0,False,True,0,29.40,27.24
29,T.Hockenson,DET,6.0,42.5,45.87,44.75,48.906667,80.0,True,True,1,45.87,47.11


In [680]:
w8_results['week']=8

### Week 10

In [681]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==10]

trailing_weeks = trailing_weeks.dropna()


prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


In [682]:
trailing_weeks[trailing_weeks['team']=='ATL'].sort_values('pYards',ascending=False)[['receiver_player_name','games','xYards_median','pYards']].round(1)

,receiver_player_name,games,xYards_median,pYards
180,K.Pitts,6.0,49.3,46.7
157,D.London,6.0,40.9,33.7
192,O.Zaccheaus,6.0,25.4,30.1
254,D.Byrd,6.0,0.0,19.7
169,K.Hodge,6.0,0.0,12.5
225,T.Allgeier,6.0,2.7,9.7
214,B.Edwards,6.0,1.7,7.8
137,A.Williams,6.0,5.3,7.1
204,P.Hesse,6.0,3.6,6.6
264,C.Huntley,6.0,0.0,4.7


In [683]:
trailing_weeks[trailing_weeks['team']=='CAR'].sort_values('pYards',ascending=False)[['receiver_player_name','games','xYards_median','pYards']].round(1)

,receiver_player_name,games,xYards_median,pYards
529,D.Moore,6.0,69.9,64.6
553,R.Anderson,6.0,40.7,37.6
517,C.McCaffrey,6.0,31.1,35.5
616,T.Marshall,6.0,25.9,26.9
577,T.Tremble,6.0,17.4,18.2
565,S.Smith,6.0,0.0,13.7
541,I.Thomas,6.0,6.9,9.6
589,G.Ricci,6.0,0.0,6.6
599,S.Sullivan,6.0,0.0,6.6
640,D.Foreman,6.0,0.0,6.4


In [684]:
player = 'V.Jefferson'

week_by_week['aDOT'] = week_by_week['air_yards']/week_by_week['targets']
all_szn = week_by_week[week_by_week['receiver_player_name']==player].sort_values('week',ascending=False)[['week','targets','aDOT','xYards','yards_gained']].round(1)

all_szn

trailing_6 = all_szn.head(6)

t_mean = trailing_6['yards_gained'].mean()
t_mean_x = trailing_6['xYards'].mean()

t_median = trailing_6['yards_gained'].median()
t_median_x = trailing_6['xYards'].median()

mean = all_szn['yards_gained'].mean()
mean_x = all_szn['xYards'].mean()

median = all_szn['yards_gained'].median()
median_x = all_szn['xYards'].median()

print('{}'.format(player))

display(all_szn.set_index('week'))


print('xYards Per Game (Trailing 6 Games): {}'.format(round(t_mean_x,1)))
print('Median xYards (Trailing 6 Games): {}\n'.format(round(t_median_x,1)))

print('Yards Per Game (Trailing 6 Games): {}'.format(round(t_mean,1)))
print('Median Yards (Trailing 6 Games): {}'.format(round(t_median,1)))



V.Jefferson


,targets,aDOT,xYards,yards_gained
week,,,,
13,4,17.8,47.2,39.0
12,6,9.5,38.0,29.0
11,5,13.0,43.8,41.0
10,3,7.7,21.1,27.0


xYards Per Game (Trailing 6 Games): 37.5
Median xYards (Trailing 6 Games): 40.9

Yards Per Game (Trailing 6 Games): 34.0
Median Yards (Trailing 6 Games): 34.0


In [685]:
w10_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards_median','gb_projection','linear_projection']]


w10_table.rename(columns={'receiver_player_name':'player'},inplace=True)

In [686]:
w10_table.set_index(['player','team'],inplace=True)

In [687]:
w10_odds = pd.read_csv('betting_lines/nfl-best-bets (5).csv').round(1)[['player','team','line']]






odds = w10_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w10_table = w10_table[w10_table['games']>1]
w10_table = w10_table.merge(odds,right_index=True,left_index=True)
w10_table = w10_table.reset_index()

w10_table['abs. diff'] = abs(w10_table['pYards'] - w10_table['line'])

w10_table = w10_table.round(1)

w10_table['Over_Picked'] =w10_table['pYards']>w10_table ['line']

w10_table.sort_values('abs. diff',ascending=True).tail(25)

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked
59,A.Kamara,NO,6.0,40.0,42.1,40.0,43.3,33.5,6.5,True
83,D.Samuel,SF,6.0,57.2,53.1,57.2,53.2,50.5,6.7,True
11,H.Bryant,CLE,6.0,12.6,8.2,12.6,13.8,19.5,6.9,False
53,T.Hill,MIA,6.0,101.8,123.8,101.8,106.9,94.5,7.3,True
43,D.Carter,LAC,6.0,30.0,28.3,30.0,29.9,37.5,7.5,False
89,M.Evans,TB,6.0,73.5,79.8,73.5,77.4,65.5,8.0,True
3,I.McKenzie,BUF,6.0,30.0,30.3,30.0,28.7,21.5,8.5,True
86,K.Juszczyk,SF,6.0,18.1,11.4,18.1,19.6,9.5,8.6,True
46,D.Adams,LV,6.0,73.5,74.4,73.5,79.1,82.5,9.0,False
57,J.Jefferson,MIN,6.0,73.5,82.3,73.5,81.0,82.5,9.0,False


In [688]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==10]


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.60s/it]


In [689]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w10_table = w10_table.merge(results)

w10_table['Over_Result'] = w10_table['yards_gained']>w10_table['line']

w10_table['Win'] = (w10_table['Over_Result'] == w10_table['Over_Picked'])*1

w10_table = w10_table[w10_table['games']>1]

w10_table['week'] = 10


In [690]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.866814101328263
-----
Root Mean Square Error - Betting Line:

29.502524233872137


In [691]:
w10_table[w10_table['team']=='IND']

,player,team,games,pYards,xYards_median,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
27,J.Taylor,IND,6.0,18.9,22.3,18.9,18.7,16.5,2.4,True,16.0,False,0,10
28,M.Pittman,IND,6.0,53.5,60.8,53.5,55.8,53.5,0.0,False,53.0,False,1,10
29,P.Campbell,IND,6.0,41.3,33.5,41.3,38.1,25.5,15.8,True,76.0,True,1,10
30,A.Pierce,IND,6.0,47.6,51.5,47.6,46.6,27.5,20.1,True,0.0,False,0,10


### Week 11

In [692]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==11]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

In [693]:
w11_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]

w11_table.rename(columns={'receiver_player_name':'player'},inplace=True)

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (8).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table[w11_table['games']>1]
w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


def o_u(over):
  if over == True:
    return 'Over'
  elif over == False:
    return 'Under'



In [694]:
tnf = w11_table[(w11_table['team']=='GB')|(w11_table['team']=='TEN')]


In [695]:
w11_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]

w11_table.rename(columns={'receiver_player_name':'player'},inplace=True)

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (9).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table[w11_table['games']>1]
w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


w11_table = w11_table.append(tnf)



In [696]:
snf = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]

snf.rename(columns={'receiver_player_name':'player'},inplace=True)

snf.set_index(['player','team'],inplace=True)

snf_odds = pd.read_csv('betting_lines/nfl-best-bets (10).csv').round(1)[['player','team','line']]






odds = snf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

snf = snf[snf['games']>1]
snf = snf.merge(odds,right_index=True,left_index=True)
snf = snf.reset_index()

snf['abs. diff'] = abs(snf['pYards'] - snf['line'])

snf = snf.round(1)

snf['Over_Picked'] =snf['pYards']>snf ['line']


primetime = snf[(snf['team']=='KC')|(snf['team']=='LAC')]

In [697]:
w11_table = w11_table.append(primetime)

In [698]:
mnf = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]

mnf.rename(columns={'receiver_player_name':'player'},inplace=True)

mnf.set_index(['player','team'],inplace=True)

mnf_odds = pd.read_csv('betting_lines/nfl-best-bets (11).csv').round(1)[['player','team','line']]






odds = mnf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

mnf = mnf[mnf['games']>1]
mnf = mnf.merge(odds,right_index=True,left_index=True)
mnf = mnf.reset_index()

mnf['abs. diff'] = abs(mnf['pYards'] - mnf['line'])

mnf = mnf.round(1)
mnf['Over_Picked'] =mnf['pYards']>mnf ['line']


mnf = mnf[(mnf['team']=='ARI')|(mnf['team']=='SF')]

In [699]:
mnf

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,J.Conner,ARI,6.0,19.6,19.4,0,20.5,18.3,20.1,19.6,19.5,0.1,True
1,R.Moore,ARI,6.0,62.3,56.6,1,60.0,67.2,56.2,62.3,59.5,2.8,True
2,T.McBride,ARI,6.0,4.7,0.9,0,0.0,1.2,4.6,4.7,19.5,14.8,False
3,B.Aiyuk,SF,6.0,63.9,56.4,1,55.1,70.8,57.4,63.9,54.5,9.4,True
4,D.Samuel,SF,6.0,52.1,56.4,1,55.1,58.8,53.1,52.1,52.5,0.4,False
5,J.Jennings,SF,6.0,26.7,23.6,1,23.4,25.7,24.8,26.7,14.5,12.2,True
6,G.Kittle,SF,6.0,40.0,42.1,0,36.2,52.0,43.5,40.0,41.5,1.5,False
7,K.Juszczyk,SF,6.0,18.1,13.4,0,11.4,25.5,19.6,18.1,7.5,10.6,True


In [700]:
w11_table = w11_table.append(mnf)

In [701]:
w11_table[(w11_table['team']=='ARI')|(w11_table['team']=='SF')]

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
0,J.Conner,ARI,6.0,19.6,19.4,0,20.5,18.3,19.6,20.1,19.5,0.1,True
1,R.Moore,ARI,6.0,62.3,56.6,1,60.0,67.2,62.3,56.2,59.5,2.8,True
2,T.McBride,ARI,6.0,4.7,0.9,0,0.0,1.2,4.7,4.6,19.5,14.8,False
3,B.Aiyuk,SF,6.0,63.9,56.4,1,55.1,70.8,63.9,57.4,54.5,9.4,True
4,D.Samuel,SF,6.0,52.1,56.4,1,55.1,58.8,52.1,53.1,52.5,0.4,False
5,J.Jennings,SF,6.0,26.7,23.6,1,23.4,25.7,26.7,24.8,14.5,12.2,True
6,G.Kittle,SF,6.0,40.0,42.1,0,36.2,52.0,40.0,43.5,41.5,1.5,False
7,K.Juszczyk,SF,6.0,18.1,13.4,0,11.4,25.5,18.1,19.6,7.5,10.6,True


In [702]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
15,D.Peoples-Jones,CLE,6.0,63.9,57.8,1,51.5,74.3,63.9,59.4,46.5,17.4,True
13,T.Higgins,CIN,6.0,57.2,52.5,1,59.5,62.2,57.2,52.3,74.5,17.3,False
18,C.Watson,GB,6.0,26.7,23.6,1,16.4,26.8,26.7,25.2,43.5,16.8,False
19,R.Cobb,GB,6.0,37.7,37.5,1,28.2,42.8,37.7,37.9,21.5,16.2,True
21,S.Watkins,GB,6.0,30.2,28.7,1,26.9,34.3,30.2,30.4,14.5,15.7,True
22,A.St. Brown,DET,6.0,65.4,67.1,1,75.8,75.0,65.4,64.3,80.5,15.1,False
2,T.McBride,ARI,6.0,4.7,0.9,0,0.0,1.2,4.7,4.6,19.5,14.8,False
22,A.Lazard,GB,6.0,63.8,67.1,1,76.1,69.0,63.8,62.1,49.5,14.3,True
18,D.Schultz,DAL,6.0,27.5,27.3,0,27.1,32.5,27.5,29.1,41.5,14.0,False
38,J.Meyers,NE,6.0,62.3,59.4,1,58.2,67.0,62.3,57.6,48.5,13.8,True


In [703]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==11]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w11_table = w11_table.merge(results)

w11_table['Over_Result'] = w11_table['yards_gained']>w11_table['line']

w11_table['Win'] = (w11_table['Over_Result'] == w11_table['Over_Picked'])*1

w11_table = w11_table[w11_table['games']>1]

w11_table['week'] = 11

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.36s/it]


In [704]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
15,D.Peoples-Jones,CLE,6.0,63.9,57.8,1,51.5,74.3,63.9,59.4,46.5,17.4,True,61.0,True,1,11
13,T.Higgins,CIN,6.0,57.2,52.5,1,59.5,62.2,57.2,52.3,74.5,17.3,False,148.0,True,0,11
56,C.Watson,GB,6.0,26.7,23.6,1,16.4,26.8,26.7,25.2,43.5,16.8,False,48.0,True,0,11
57,R.Cobb,GB,6.0,37.7,37.5,1,28.2,42.8,37.7,37.9,21.5,16.2,True,73.0,True,1,11
59,S.Watkins,GB,6.0,30.2,28.7,1,26.9,34.3,30.2,30.4,14.5,15.7,True,0.0,False,0,11
22,A.St. Brown,DET,6.0,65.4,67.1,1,75.8,75.0,65.4,64.3,80.5,15.1,False,76.0,False,1,11
74,T.McBride,ARI,6.0,4.7,0.9,0,0.0,1.2,4.7,4.6,19.5,14.8,False,14.0,False,1,11
60,A.Lazard,GB,6.0,63.8,67.1,1,76.1,69.0,63.8,62.1,49.5,14.3,True,57.0,True,1,11
18,D.Schultz,DAL,6.0,27.5,27.3,0,27.1,32.5,27.5,29.1,41.5,14.0,False,22.0,False,1,11
38,J.Meyers,NE,6.0,62.3,59.4,1,58.2,67.0,62.3,57.6,48.5,13.8,True,52.0,True,1,11


In [705]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.866814101328263
-----
Root Mean Square Error - Betting Line:

29.502524233872137


### Week 12

In [706]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==12]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

w12_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w12_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (12).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table[w12_table['games']>1]
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 



In [707]:
tg = w12_table[w12_table['team']!='CAR'].copy()

In [708]:
prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']

w12_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w12_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (13).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table[w12_table['games']>1]
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']

w12_table = w12_table.append(tg)


In [709]:


mnf12 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
mnf12.rename(columns={'receiver_player_name':'player'},inplace=True)
 
mnf12.set_index(['player','team'],inplace=True)
 
mnf12_odds = pd.read_csv('betting_lines/nfl-best-bets (14).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf12 = mnf12.merge(odds,right_index=True,left_index=True)
mnf12 = mnf12.reset_index()
 
mnf12['abs. diff'] = abs(mnf12['pYards'] - mnf12['line'])
 
mnf12 = mnf12.round(1)
 
mnf12['Over_Picked'] =mnf12['pYards']>mnf12 ['line']

w12_table = w12_table.append(mnf12)


In [710]:
w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])
w12_table[(w12_table['abs. median diff']>=10)&(w12_table['abs. diff']>=10)]

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked,abs. median diff
11,A.Cooper,CLE,6.0,65.4,65.4,1,75.7,78.3,65.4,64.6,51.5,13.9,True,24.2
14,K.Hinton,DEN,6.0,25.2,16.3,1,13.6,30.0,25.2,22.6,35.5,10.3,False,21.9
19,A.Lazard,GB,6.0,59.4,70.3,1,76.1,59.2,59.4,60.3,46.5,12.9,True,29.6
44,T.Hill,MIA,6.0,80.1,100.6,1,104.3,111.8,80.1,94.3,92.5,12.4,False,11.8
49,G.Wilson,NYJ,6.0,37.7,36.3,1,29.3,46.3,37.7,38.6,53.5,15.8,False,24.2
51,A.Brown,PHI,6.0,57.2,59.3,1,51.5,63.5,57.2,56.3,71.0,13.8,False,19.5
59,T.Lockett,SEA,6.0,44.5,44.0,1,45.5,56.3,44.5,46.0,63.5,19.0,False,18.0
67,M.Evans,TB,6.0,71.3,82.3,1,77.2,72.7,71.3,71.1,60.5,10.8,True,16.7
2,G.Davis,BUF,6.0,71.3,70.9,1,71.8,79.0,71.3,67.7,58.5,12.8,True,13.3


In [711]:
mnf12

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,J.Taylor,IND,6.0,17.2,18.9,0,19.0,12.3,17.7,17.2,13.5,3.7,True
1,M.Alie-Cox,IND,6.0,6.0,3.1,0,0.0,4.5,6.9,6.0,13.5,7.5,False
2,M.Pittman,IND,6.0,63.8,65.2,1,61.1,65.8,60.1,63.8,67.5,3.7,False
3,P.Campbell,IND,6.0,48.0,47.0,1,50.9,54.7,46.9,48.0,48.5,0.5,False
4,A.Pierce,IND,6.0,34.0,36.2,1,42.0,33.7,34.0,34.0,35.5,1.5,False
5,D.Johnson,PIT,6.0,48.8,52.6,1,46.0,41.5,45.1,48.8,45.5,3.3,True
6,G.Pickens,PIT,6.0,38.6,36.1,1,37.3,47.7,38.9,38.6,43.5,4.9,False
7,N.Harris,PIT,6.0,18.2,19.2,0,19.6,15.0,18.8,18.2,17.5,0.7,True
8,P.Freiermuth,PIT,6.0,49.9,55.7,0,58.4,57.3,52.2,49.9,48.5,1.4,True
9,Z.Gentry,PIT,6.0,12.6,12.3,0,7.5,9.7,13.4,12.6,7.5,5.1,True


In [712]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==12]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w12_table = w12_table.merge(results)

w12_table['Over_Result'] = w12_table['yards_gained']>w12_table['line']

w12_table['Win'] = (w12_table['Over_Result'] == w12_table['Over_Picked'])*1

w12_table = w12_table[w12_table['games']>1]

w12_table['week'] = 12

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.18s/it]


### Week 13

In [713]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==13]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


w13_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','gb_projection','linear_projection']]
 
w13_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (16).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['pYards'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['pYards']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf = w13_table.copy()

In [714]:
w13_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
w13_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (20).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['gb_projection'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['gb_projection']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf.append(w13_table[(w13_table['team']!='BUF')&(w13_table['team']!='NE')])

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,gb_projection,linear_projection,line,abs. diff,Over_Picked
0,D.Knox,BUF,6.0,35.0,33.0,0,27.6,36.0,35.0,33.2,32.5,2.5,True
1,D.Singletary,BUF,6.0,17.2,16.0,0,12.9,13.5,17.2,16.6,11.5,5.7,True
2,G.Davis,BUF,6.0,53.5,66.8,1,67.6,56.8,53.5,57.7,48.5,5.0,True
3,I.McKenzie,BUF,6.0,30.0,28.8,1,21.9,27.0,30.0,27.9,32.5,2.5,False
4,S.Diggs,BUF,6.0,73.5,97.1,1,108.8,100.3,73.5,88.5,81.5,8.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,D.Henry,TEN,6.0,19.2,11.4,0,13.2,26.2,19.2,18.8,14.5,4.7,True
115,C.Samuel,WAS,6.0,29.6,26.5,1,31.7,34.3,29.6,29.3,30.5,0.9,False
116,J.Dotson,WAS,6.0,23.5,25.7,1,25.7,23.2,23.5,25.0,19.5,4.0,True
117,L.Thomas,WAS,6.0,18.2,17.2,0,17.9,17.2,18.2,18.5,23.5,5.3,False


In [715]:
w13_table = w13_table[w13_table['team']!='IND']

In [716]:
w13_table = w13_table[w13_table['team']!='DAL']

w13_table = w13_table[w13_table['team']!='NO']

w13_table = w13_table[w13_table['team']!='TB']

In [717]:
w13 = trailing_weeks.sort_values('pYards',ascending=False)[['full_name','team','pYards','xYards_median','xYards/game','Yards/game','linear_projection']].round(1)

#w13[(w13['team']=='IND')|(w13['team']=='DAL')]

In [718]:
snf13 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
snf13.rename(columns={'receiver_player_name':'player'},inplace=True)
 
snf13.set_index(['player','team'],inplace=True)
 
snf13_odds = pd.read_csv('betting_lines/nfl-best-bets (21).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf13 = snf13.merge(odds,right_index=True,left_index=True)
snf13 = snf13.reset_index()
 
snf13['abs. diff'] = abs(snf13['gb_projection'] - snf13['line'])
 
snf13 = snf13.round(1)
 
snf13['Over_Picked'] =snf13['gb_projection']>snf13 ['line']

In [719]:
mnf13 = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
mnf13.rename(columns={'receiver_player_name':'player'},inplace=True)
 
mnf13.set_index(['player','team'],inplace=True)
 
mnf13_odds = pd.read_csv('betting_lines/nfl-best-bets (23).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf13 = mnf13.merge(odds,right_index=True,left_index=True)
mnf13 = mnf13.reset_index()
 
mnf13['abs. diff'] = abs(mnf13['gb_projection'] - mnf13['line'])
 
mnf13 = mnf13.round(1)
 
mnf13['Over_Picked'] = mnf13['gb_projection']>mnf13 ['line']

In [720]:
#mnf13

In [721]:
w13_table = w13_table.append(snf13)

w13_table = w13_table.append(mnf13)

In [722]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==13]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w13_table = w13_table.merge(results)

w13_table['Over_Result'] = w13_table['yards_gained']>w13_table['line']

w13_table['Win'] = (w13_table['Over_Result'] == w13_table['Over_Picked'])*1

w13_table = w13_table[w13_table['games']>1]

w13_table['week'] = 13

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.43s/it]


In [723]:
#w13_table[w13_table['abs. diff']>=10].sort_values('abs. diff',ascending=False).head(50)

### Week 14

In [751]:
trailing_weeks = weekly_receivers.reset_index()
trailing_weeks = trailing_weeks_all[trailing_weeks_all['target_week']==14]

trailing_weeks = trailing_weeks.dropna()



prediction_X = trailing_weeks[features]

prediction_X_linear = trailing_weeks[linear_features]

trailing_weeks['gb_projection']=in_season_model.predict(prediction_X)

#trailing_weeks['pYards']=in_season_model.predict(prediction_X)

trailing_weeks['linear_projection']=linear_model.predict(prediction_X_linear)

trailing_weeks['pYards']=trailing_weeks['gb_projection']


w14_table = trailing_weeks[['receiver_player_name','team','games','pYards','xYards/game','WR','xYards_median','Yards/game','linear_projection','gb_projection']]
 
w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (24).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,right_index=True,left_index=True)
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['gb_projection'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['gb_projection']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
tnf = w14_table[(w14_table['team']=='LA')|(w14_table['team']=='LV')]


tnf.append(w14_table[(w14_table['team']!='LA')&(w14_table['team']!='LV')])

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked
0,D.Adams,LV,6.0,85.2,106.7,1,104.0,111.2,97.1,85.2,89.5,4.3,False
1,J.Jacobs,LV,6.0,28.0,27.9,0,25.2,31.7,29.1,28.0,20.5,7.5,True
2,M.Hollins,LV,6.0,48.8,57.8,1,61.5,41.5,47.7,48.8,45.5,3.3,True
3,T.Higbee,LA,6.0,24.5,25.7,0,27.7,24.5,25.4,24.5,35.0,10.5,False
4,V.Jefferson,LA,6.0,23.5,25.0,1,29.5,22.7,24.4,23.5,37.5,14.0,False


In [753]:
pp=trailing_weeks.copy()

#pp = trailing_weeks[trailing_weeks['full_name']=="D.J. Moore"]


pp[['full_name','pYards','linear_projection','xYards/game','Yards/game']].sort_values('pYards',ascending=False).head(50)

,full_name,pYards,linear_projection,xYards/game,Yards/game
2473,Tyreek Hill,85.200581,98.411511,107.899956,113.000000
2302,Davante Adams,85.200581,97.136479,106.666067,111.166667
789,Ja'Marr Chase,73.466269,74.368719,78.947001,86.500000
2545,Justin Jefferson,73.466269,83.441121,84.705312,103.833333
2084,Cooper Kupp,73.466269,71.803523,72.339423,88.666667
105,DeAndre Hopkins,73.466269,80.286583,84.210647,95.666667
996,CeeDee Lamb,73.466269,73.099417,76.432803,86.500000
1249,Amon-Ra St. Brown,73.466269,74.347611,74.677939,92.500000
3870,Terry McLaurin,73.466269,71.253936,74.421255,84.166667
458,Stefon Diggs,73.466269,83.338154,93.542979,91.000000


In [726]:
#moore

In [756]:
player__ = 'T.Higbee'


player_analyzer2 = player_analyzer.copy()

player_by_week = player_analyzer2[player_analyzer2['receiver_player_name']==player__]

#team = 'PHI'

#player_by_week = player_by_week[player_by_week['team']==team]


player_disp = player_by_week[['full_name','team','week','offense_snaps','xYards','yards_gained','targets','target_share','aDOT','complete_pass','cp','yards_after_catch','xyac_mean_yardage']].round(2).drop_duplicates()

player_mean = player_disp.tail(6)['xYards'].mean()

player_sd = player_disp.tail(6)['xYards'].std()

player_disp

,full_name,team,week,offense_snaps,xYards,yards_gained,targets,target_share,aDOT,complete_pass,cp,yards_after_catch,xyac_mean_yardage
0,Tyler Higbee,LA,1.0,63.0,63.20,39.0,11.0,0.27,1.73,5.0,8.43,27.0,28.85
1,Tyler Higbee,LA,2.0,61.0,57.13,71.0,9.0,0.26,6.33,7.0,6.40,40.0,30.64
2,Tyler Higbee,LA,3.0,45.0,24.98,61.0,4.0,0.16,0.00,4.0,3.32,61.0,32.88
3,Tyler Higbee,LA,4.0,69.0,76.12,73.0,14.0,0.29,3.71,10.0,9.85,44.0,51.36
4,Tyler Higbee,LA,5.0,63.0,62.81,46.0,10.0,0.24,3.70,7.0,7.71,17.0,33.00
5,Tyler Higbee,LA,6.0,51.0,14.22,7.0,2.0,0.06,6.00,1.0,1.51,5.0,5.51
7,Tyler Higbee,LA,8.0,40.0,33.52,15.0,6.0,0.19,-0.17,2.0,5.07,15.0,14.30
8,Tyler Higbee,LA,9.0,45.0,0.00,0.0,0.0,NaN,NaN,0.0,0.00,0.0,0.00
9,Tyler Higbee,LA,10.0,48.0,43.55,73.0,8.0,0.22,2.88,8.0,6.05,50.0,43.45
10,Tyler Higbee,LA,11.0,59.0,55.19,45.0,8.0,0.30,6.50,4.0,5.94,11.0,20.90


## Overall Results

In [728]:
szn_results = w8_results.append(w9_table).append(w10_table).append(w11_table).append(w12_table).append(w13_table)

szn_results['abs_diff'] = abs(szn_results['pYards']-szn_results['line'])

szn_results['smash'] = szn_results['abs_diff']>=10

smash_df = szn_results[szn_results['smash']==True]


In [729]:
print('R Values:\n')


print('Yards and projection: {}'.format(round(szn_results['pYards'].corr(szn_results['yards_gained']),3)))
print('Yards and betting line: {}'.format(round(szn_results['line'].corr(szn_results['yards_gained']),3)))
print('Projection and betting line: {}'.format(round(szn_results['pYards'].corr(szn_results['line']),3)))


R Values:

Yards and projection: 0.546
Yards and betting line: 0.603
Projection and betting line: 0.941


In [730]:


print('Overall Record: {}-{} ({}%)'.format(szn_results['Win'].sum(),szn_results.shape[0]-szn_results['Win'].sum(),round(szn_results['Win'].sum()/szn_results.shape[0]*100,1)))

print('Plays Record (10+ yard difference): {}-{} ({}%)'.format(smash_df['Win'].sum(),smash_df.shape[0]-smash_df['Win'].sum(),round(smash_df['Win'].sum()/smash_df.shape[0]*100,1)))

print('\n-----\n')
 
mae = mean_absolute_error(szn_results['pYards'], szn_results['yards_gained'])
    
print("Mean Absolute Error - Projections:")
print('{}\n'.format(round(mae,1)))


mae = mean_absolute_error(szn_results['yards_gained'],szn_results['line'])
 
print("Mean Absolute Error - Betting line:")
print(round(mae,1))

Overall Record: 296-289 (50.6%)
Plays Record (10+ yard difference): 51-46 (52.6%)

-----

Mean Absolute Error - Projections:
22.2

Mean Absolute Error - Betting line:
21.4


In [731]:
MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:")
print('{}\n'.format(round(RMSE,1)))


MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['line'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting line:")
print(round(RMSE,1))

Root Mean Square Error - Projections:
29.7

Root Mean Square Error - Betting line:
28.5


In [732]:
print('WEEKLY RESULTS')

print('\nOverall')
for week in szn_results['week'].unique():
    w = szn_results[szn_results['week']==week]['Win'].sum()
    p = szn_results[szn_results['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))


print('\nPlays')

for week in smash_df['week'].unique():
    w = smash_df[smash_df['week']==week]['Win'].sum()
    p = smash_df[smash_df['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))

WEEKLY RESULTS

Overall
Week 8: 52-48 (52.0%)
Week 9: 41-45 (47.7%)
Week 10: 44-50 (46.8%)
Week 11: 46-33 (58.2%)
Week 12: 54-51 (51.4%)
Week 13: 59-62 (48.8%)

Plays
Week 8: 8-7 (53.3%)
Week 9: 8-7 (53.3%)
Week 10: 3-7 (30.0%)
Week 11: 11-7 (61.1%)
Week 12: 10-10 (50.0%)
Week 13: 11-8 (57.9%)


In [733]:
w13_table.sort_values('abs. diff',ascending=False).head(19)

,player,team,games,pYards,xYards/game,WR,xYards_median,Yards/game,linear_projection,gb_projection,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
51,D.Adams,LV,6.0,73.5,100.6,1,95.0,97.5,89.2,73.5,95.5,22.0,False,177.0,True,0,13
66,G.Wilson,NYJ,6.0,44.5,41.0,1,42.8,57.7,45.0,44.5,62.5,18.0,False,162.0,True,0,13
82,T.Lockett,SEA,6.0,48.0,46.3,1,45.7,50.3,45.0,48.0,63.5,15.5,False,128.0,True,0,13
113,J.Landry,NO,6.0,26.8,30.7,1,27.5,24.0,27.8,26.8,40.5,13.7,False,14.0,False,1,13
46,T.Kelce,KC,6.0,71.3,75.3,0,63.2,94.2,75.2,71.3,84.5,13.2,False,56.0,False,1,13
16,D.Peoples-Jones,CLE,6.0,60.6,47.0,1,41.0,67.0,51.3,60.6,47.5,13.1,True,44.0,False,0,13
13,T.Boyd,CIN,6.0,55.5,46.7,1,39.1,60.2,48.7,55.5,42.5,13.0,True,60.0,True,1,13
17,H.Bryant,CLE,6.0,18.2,18.1,0,14.4,16.3,18.7,18.2,30.5,12.3,False,6.0,False,1,13
69,A.Brown,PHI,6.0,62.3,57.7,1,48.8,65.8,56.3,62.3,74.5,12.2,False,119.0,True,0,13
12,S.Perine,CIN,6.0,19.6,17.5,0,14.1,19.8,19.6,19.6,31.5,11.9,False,49.0,True,0,13


In [734]:
szn_results['Over_Picked'].value_counts()

True     346
False    239
Name: Over_Picked, dtype: int64

In [735]:
szn_results['Over_Result'].value_counts()

True     309
False    276
Name: Over_Result, dtype: int64

In [736]:
over_w = szn_results[szn_results['Over_Picked']==1]['Win'].mean()
under_w = szn_results[szn_results['Over_Picked']==0]['Win'].mean()

print('Overs: {}%'.format(round(over_w*100,1)))
print('Under: {}%'.format(round(under_w*100,1)))

Overs: 52.9%
Under: 47.3%


In [737]:
over_w = smash_df[smash_df['Over_Picked']==1]['Win'].mean()
under_w = smash_df[smash_df['Over_Picked']==0]['Win'].mean()

print('Smash Overs: {}%'.format(round(over_w*100,1)))
print('Smash Under: {}%'.format(round(under_w*100,1)))

Smash Overs: 51.1%
Smash Under: 53.8%


In [738]:
szn_results['the_blend'] = (szn_results['xYards_median']+szn_results['pYards'])/2

szn_results[['pYards','yards_gained','xYards/game','line','the_blend','xYards_median','gb_projection','linear_projection']].corr().sort_values('yards_gained',ascending=False)

,pYards,yards_gained,xYards/game,line,the_blend,xYards_median,gb_projection,linear_projection
yards_gained,0.545616,1.000000,0.551937,0.602547,0.556503,0.554497,0.545616,0.556812
line,0.941316,0.602547,0.933044,1.000000,0.939313,0.918464,0.941316,0.943952
linear_projection,0.987990,0.556812,0.987852,0.943952,0.988415,0.968646,0.987990,1.000000
the_blend,0.987188,0.556503,0.989386,0.939313,1.000000,0.990609,0.987188,0.988415
xYards_median,0.956102,0.554497,0.978067,0.918464,0.990609,1.000000,0.956102,0.968646
xYards/game,0.976096,0.551937,1.000000,0.933044,0.989386,0.978067,0.976096,0.987852
pYards,1.000000,0.545616,0.976096,0.941316,0.987188,0.956102,1.000000,0.987990
gb_projection,1.000000,0.545616,0.976096,0.941316,0.987188,0.956102,1.000000,0.987990


In [739]:
szn_results[szn_results['gb_projection'].isna()==True]

,player,team,games,line,pYards,xYards_median,xYards/game,yards_gained,Over_Picked,Over_Result,...,gb_projection,linear_projection,week,abs. diff,WR,Yards/game,abs. median diff,abs_diff,smash,the_blend
